In [16]:
# ========================
# library
# ========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pickle
import glob
from tqdm import tqdm
%matplotlib inline

In [17]:
# ========================
# constant
# ========================
DEFOG_META_PATH = "../data/defog_metadata.csv"
DEFOG_FOLDER = "../data/train/notype/*.csv"

In [18]:
# ========================
# settings
# ========================
fe = "073"
ex = "153"
if not os.path.exists(f"../output/fe/fe{fe}"):
    os.makedirs(f"../output/fe/fe{fe}")
    os.makedirs(f"../output/fe/fe{fe}/save")

In [19]:
data_list = glob.glob(DEFOG_FOLDER)

In [20]:
meta = pd.read_parquet("../output/fe/fe039/fe039_defog_meta.parquet")

In [21]:
cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff', 'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
       'AccAP_lag_diff', 'AccAP_lead_diff']
target_use_cols = ["Event"]
target_cols = ["StartHesitation","Turn","Walking"]
seq_len = 5000
shift = 2500
offset = 1250

In [22]:
for fold in range(5):
    print(fold)
    pred = pd.read_parquet(f"../output/exp/ex{ex}/ex{ex}_notype_{fold}_pred_10000.parquet")
    target_array = []
    for i,s in tqdm(zip(meta["Id"].values,
                   meta["sub_id"].values)):
        path = f"../data/train/notype/{i}.csv"
        if path in data_list:
            df = pd.read_csv(path)
            df_ = pred[pred["Id"] == i].reset_index(drop=True)
            df = df.merge(df_,how="left",on="Time")
            df["target_max"] = np.argmax(df[["StartHesitation","Turn","Walking"]].values,axis=1)
            
            df.loc[df["target_max"] == 0,"StartHesitation"] = 1
            df.loc[df["target_max"] == 0,["Turn","Walking"]] = 0
            
            df.loc[df["target_max"] == 1,"Turn"] = 1
            df.loc[df["target_max"] == 1,["StartHesitation","Walking"]] = 0
            
            df.loc[df["target_max"] == 2,"Walking"] = 1
            df.loc[df["target_max"] == 2,["StartHesitation","Turn"]] = 0

            df.loc[df["Event"] == 0,["StartHesitation","Turn","Walking"]] = 0

            df["valid"] = df["Valid"] & df["Task"]
            df["valid"] = df["valid"].astype(int)
            batch = (len(df)-1) // shift
            target = df[target_cols].values
            target_array_ = np.zeros([batch,seq_len,3])
            for n,b in enumerate(range(batch)):
                if b == (batch - 1):
                    target_ = target[b*shift : ]
                    target_array_[b,:len(target_),:] = target_
                elif b == 0:
                    target_ = target[b*shift:b*shift + seq_len]
                    target_array_[b,:,:] = target_
                else:
                    target_ = target[b*shift:b*shift + seq_len]
                    target_array_[b,:,:] = target_

            target_array.append(target_array_)
    target_array = np.concatenate(target_array,axis=0)
    np.save(f"../output/fe/fe{fe}/fe{fe}_target_array_{fold}.npy",target_array)

0


137it [00:46,  2.95it/s]


1


137it [00:41,  3.27it/s]


2


137it [00:45,  3.02it/s]


3


137it [00:46,  2.95it/s]


4


137it [00:46,  2.92it/s]


In [23]:
df

,Time,AccV,AccML,AccAP,Event,Valid,Task,Id,StartHesitation,Turn,Walking,target_max,valid
0,0,-0.943677,-0.291860,0.267011,0,False,False,e8e530a4f9,0.0,0.0,0.0,2,0
1,1,-0.944334,-0.290016,0.263428,0,False,False,e8e530a4f9,0.0,0.0,0.0,2,0
2,2,-0.942461,-0.288839,0.262152,0,False,False,e8e530a4f9,0.0,0.0,0.0,2,0
3,3,-0.942056,-0.291109,0.264220,0,False,False,e8e530a4f9,0.0,0.0,0.0,2,0
4,4,-0.942278,-0.291462,0.261209,0,False,False,e8e530a4f9,0.0,0.0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181615,181615,-0.985195,-0.244639,0.007996,0,False,False,e8e530a4f9,0.0,0.0,0.0,1,0
181616,181616,-0.984721,-0.243622,0.004757,0,False,False,e8e530a4f9,0.0,0.0,0.0,1,0
181617,181617,-0.985422,-0.240600,0.006233,0,False,False,e8e530a4f9,0.0,0.0,0.0,1,0
181618,181618,-0.985448,-0.239702,0.009785,0,False,False,e8e530a4f9,0.0,0.0,0.0,1,0


In [24]:
pred

,Time,Id,StartHesitation,Turn,Walking
0,0,02ab235146,0.000133,0.001808,0.001950
1,1,02ab235146,0.000120,0.001606,0.001755
2,2,02ab235146,0.000113,0.001452,0.001537
3,3,02ab235146,0.000110,0.001385,0.001491
4,4,02ab235146,0.000110,0.001378,0.001517
...,...,...,...,...,...
10251109,181615,e8e530a4f9,0.000159,0.000660,0.000479
10251110,181616,e8e530a4f9,0.000158,0.000661,0.000478
10251111,181617,e8e530a4f9,0.000157,0.000662,0.000478
10251112,181618,e8e530a4f9,0.000157,0.000662,0.000477
